In [51]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    path_to_file = '/content/gdrive/My Drive/Github/CS4248-Project/Jarrod/'
    print(path_to_file)
    # move to Google Drive directory
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Github/CS4248-Project/Jarrod/
/content/gdrive/My Drive/Github/CS4248-Project/Jarrod


In [3]:
!pip install stanfordcorenlp

In [33]:
import pandas as pd
import os
from tqdm import tqdm
from stanfordcorenlp import StanfordCoreNLP

nlp = StanfordCoreNLP(r'stanford-corenlp-4.4.0')

sentence = 'My dog likes to shake his stuffed chikadee toy. Hey there how are you doing!'
print ('Constituency Parsing:', nlp.parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Constituency Parsing: (ROOT
  (S
    (NP (PRP$ My) (NN dog))
    (VP (VBZ likes)
      (S
        (VP (TO to)
          (VP (VB shake)
            (NP (PRP$ his) (VBN stuffed) (NN chikadee) (NN toy))))))
    (. .)))


In [68]:
neg_data = open("all_neg.txt", "r")
neg_lines = neg_data.readlines()
print (len(neg_lines))
neg_data.close()

12500


In [69]:
pos_data = open("all_pos.txt", "r")
pos_lines = pos_data.readlines()
print (len(pos_lines))
pos_data.close()

12500


In [70]:
test_neg_data = open("test_neg.txt", "r")
test_neg_lines = test_neg_data.readlines()
print (len(test_neg_lines))
test_neg_data.close()

12500


In [71]:
test_pos_data = open("test_pos.txt", "r")
test_pos_lines = test_pos_data.readlines()
print (len(test_pos_lines))
test_pos_data.close()

12500


In [28]:
print(pos_lines[0])
print(neg_lines[0])

For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.

Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.



Now, we need to clean up the data to remove any line breaks etc.

Remove any websites and emails also.

regex for Urls taken from : https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url


In [72]:
import re
import string

neg_lines_replaced = [re.sub("<br />|(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"," ",line) for line in neg_lines]
pos_lines_replaced = [re.sub("<br />|(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"," ",line) for line in pos_lines]
test_neg_replaced = [re.sub("<br />|(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"," ",line) for line in test_neg_lines]
test_pos_replaced = [re.sub("<br />|(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"," ",line) for line in test_pos_lines]

After cleaning, we want to create the trees that will help to store the data for treeLSTM/recursive NNs

For each review, note that we will assign the sentiment value to each sentence based on the overall sentiment of the review by its label. If its negative, then each sentence will be -1; +1 for positive.

There are other ways to tackle this also in the process to build a tree and parse the data. For example, rather than have each sentence in the review be -1/1, it could be an average. If the review has 4 sentences and it reads as negative, then we know that each sentence will represent 1/4 of the overall movie sentiment. This is something else that can be tried with.

In [74]:
import nltk

nltk.download('punkt')

print(neg_lines_replaced[0])
print(type(neg_lines_replaced[0]))

def text_to_sentences(lines):
  result = []
  for line in lines:
    sents = nltk.tokenize.sent_tokenize(line)
    result = result + sents
  return result

neg_lines_sentences = text_to_sentences(neg_lines_replaced)
print(neg_lines_sentences[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.  Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.

<class 'str'>
Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.


In [43]:
pos_lines_sentences = text_to_sentences(pos_lines_replaced)
print(len(pos_lines_sentences))
print(len(neg_lines_sentences))

151999
159297


In [73]:
test_neg_lines_sentences = text_to_sentences(test_neg_replaced)
test_pos_lines_sentences = text_to_sentences(test_pos_replaced)
print(len(test_pos_lines_sentences))
print(len(test_neg_lines_sentences))

148144
156816


The next part is a sample tree structure of the sentences in the sentences array to be passed into the TreeLSTM / Recursive NN for learning.

In [47]:
nlp = StanfordCoreNLP(r'stanford-corenlp-4.4.0')

print(nlp.parse(neg_lines_sentences[0]))

nlp.close()



(ROOT
  (S
    (S
      (VP (VBG Working)
        (PP (IN with)
          (NP
            (NP (CD one))
            (PP (IN of)
              (NP (DT the) (JJS best) (NNP Shakespeare) (NNS sources)))))))
    (, ,)
    (NP (DT this) (NN film))
    (VP (VBZ manages)
      (S
        (VP (TO to)
          (VP (VB be)
            (ADJP (JJ creditable))
            (SBAR (IN to)
              (S
                (NP (PRP it))
                (VP (VBZ 's)
                  (NP (NN source)))))
            (, ,)
            (PP (IN whilst)
              (S
                (ADVP (RB still))
                (VP (VBG appealing)
                  (PP (IN to)
                    (NP (DT a) (JJR wider) (NN audience))))))))))
    (. .)))


In [48]:
!pip install stanza

     |████████████████████████████████| 432 kB 4.1 MB/s 
     |████████████████████████████████| 174 kB 48.6 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=43878448d9e65d7f9af66dcb52b40a0a066569a28fda77f450924d80d2bd5241
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


Here we can used the model developed by Stanford team to check what would the the outcome of the individual sentences in our movie review sentiment, and then we would develop a specific matrix to determine the overall efficacy of the stanford sentiment model.

In [53]:
import stanza
stanza.install_corenlp()


In [56]:
stanza.download('en')


INFO:stanza:Downloading default packages for language: en (English)...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


We can also use the stanza module to do our own constituency parsing that can be passed into a TreeLSTM model for training. Here, I am only using a benchmark of 100 sentences in the entire sentence corpus to understand the resource-constraints of preparing data for a TreeLSTM model. Note that with simple cleaning of the data the negative_sentences corpus alone already has a size of >150000 sentences.

In [65]:
from stanza.server import CoreNLPClient


nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')

import time

neg_constituents_map = []
i = 0

start = time.time()

for sentence in neg_lines_sentences[:100]:
  doc = nlp(sentence)
  for sentence in doc.sentences:
    neg_constituents_map.append(sentence.constituency)

end = time.time()
print(end - start , ' seconds')
print(neg_constituents_map[0])

INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


20.252846002578735  seconds
(ROOT (S (S (VP (VBG Working) (PP (IN with) (NP (NP (CD one)) (PP (IN of) (NP (DT the) (JJS best) (NNP Shakespeare) (NNS sources))))))) (, ,) (NP (DT this) (NN film)) (VP (VBZ manages) (S (VP (TO to) (VP (VB be) (ADJP (JJ creditable) (PP (IN to) (NP (NP (PRP it) (POS 's)) (NN source)))) (, ,) (PP (IN whilst) (S (ADVP (RB still)) (ADJP (JJ appealing) (PP (IN to) (NP (DT a) (JJR wider) (NN audience)))))))))) (. .)))


Here what we want to do is to pass our data through the model and evaluate the results. Before passing thru the model, we will first develop a sentence array for each of the individual reviews.

In [76]:
def text_to_sentences_arr(lines):
  result = []
  for line in lines:
    sents = nltk.tokenize.sent_tokenize(line)
    result.append(sents)
  return result


test_neg_2d = text_to_sentences_arr(test_neg_replaced)
test_pos_2d = text_to_sentences_arr(test_pos_replaced)

print(test_neg_2d[0])


['Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick.', "It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook.", 'These three actors mannage to entertain us no matter what the movie, it seems.', 'The plot for the movie shows potential, but one gets the impression in watching the film that it was not pulled off as well as it could have been.', 'The fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things.', 'The movie is worth a view- if for nothing more than entertaining performances by Rickman, Thompson, and Holbrook.']


In [78]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

import time

neg_test_sentiments_map = []
i=0
start = time.time()

for review in test_neg_2d[:100]:
  doc = nlp(' '.join(review))
  review_sentiment_arr = []
  for sentence in doc.sentences:
    review_sentiment_arr.append(sentence.sentiment)
  neg_test_sentiments_map.append(review_sentiment_arr)
  print(i)
  i=i+1
end = time.time()
print(end - start , ' seconds')
print(neg_test_sentiments_map[0])

INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| sentiment | sstplus  |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: sentiment
INFO:stanza:Done loading processors!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
42.43351197242737  seconds
[2, 1, 1, 0, 0, 2]


After obtaining the results of the model, we need to make sense of the results. As a result of doing sentence sentiment analysis, we need to identify a good heuristic for us to obtain the overall sentiment of the review. Are we able to then use a regular neural network to do this checking on what might be the best formula?

Possible formulas to use include

1. Direct summation and averaging
2. Number of words in the sentence - requires more data augmentation.
3. Removal of stop words?

heuristics to include
1. threshold for which u identify a sentiment as positive or negative

For simplicity, we shall consider the first case first, that is we will use the direct sum of all the values in the array and then divide over to get an average score of the review. If the score is < 0 then the review is negative in sentiment and vice versa.

In [80]:
threshold = 0
def change_sentiment_value(i):
  if i == 2:
    return 1
  elif i == 0:
    return -1
  else:
    return 0

def get_sentiments(review):
  average_sentiment = sum([change_sentiment_value(i) for i in review])/len(review)
  if average_sentiment > threshold:
    return 1
  elif average_sentiment < threshold:
    return -1
  else:
    return 0

neg_sentiment_results = [get_sentiments(review) for review in neg_test_sentiments_map]
print(neg_sentiment_results[:20])

[0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


It is easy to evaluate the accuracy of the model by looking at the number of values that are not the intended values for the test set.

Take for example if we look at the the neg_sentiment_results, which stores the result of each review after running through the Stanford model and producing the result.

We simply count the number N of non-(-1)s in the array and then use the formula of [1-(N/len(array))] * 100% to get the eventual accuracy of the model.

In [87]:
neg_model_accuracy = (neg_sentiment_results.count(-1)/len(neg_sentiment_results))*100
print(neg_sentiment_results.count(-1))
print(len(neg_sentiment_results))
print(neg_model_accuracy)
print(neg_sentiment_results)

90
100
90.0
[0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 1, -1, -1, -1, -1, 0, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
